[Define Agents](https://chatgpt.com/share/670fa620-5928-800c-82e0-e6432665ae93): In the context of large language models (LLMs), agents are autonomous systems or programs that leverage the natural language understanding and generation capabilities of LLMs to perform tasks, make decisions, and interact with environments or users. These agents use LLMs as their core reasoning and communication component to:

- Interpret Instructions: Understand and process natural language commands or queries from users.
- Plan Actions: Generate sequences of actions or responses based on the given instructions or goals.
- Interact with Environments: Engage with software interfaces, databases, or other systems to retrieve or manipulate information.
- Execute Tasks: Perform specific functions such as data analysis, content creation, customer support, or automation tasks.

By utilizing LLMs, these agents can handle complex language-related tasks that require understanding context, managing dialogues, and generating human-like responses, making them valuable in applications like virtual assistants, chatbots, and automated customer service.

- [ReAct: Synergizing Reasoning and Acting in Language Models](https://github.com/ysymyth/ReAct)
- [AutoGen](https://microsoft.github.io/autogen/0.2/docs/Use-Cases/agent_chat)
- [CrewAI](https://docs.crewai.com/introduction )


[Agents simulating human behavior](https://www.youtube.com/watch?v=ZdoU9vI2yCg)

- [Langchain Agent Types](https://python.langchain.com/v0.1/docs/modules/agents/agent_types/)
- [Llama-index Agent Types](https://docs.llamaindex.ai/en/stable/module_guides/deploying/agents/)
- [Llama Stack](https://github.com/meta-llama/llama-stack-apps)

# BabyAGI with Tools

This notebook builds on top of [baby agi](https://github.com/yoheinakajima/babyagi/tree/main), but shows how you can swap out the execution chain. The previous execution chain was just an LLM which made stuff up. By swapping it out with an agent that has access to tools, we can hopefully get real reliable information

In [ ]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.5 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [ ]:
!pip install langchain_experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.1/208.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.0 MB/s eta 0:00:00


In [ ]:
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 6.4 MB/s eta 0:00:00


## Install and Import Required Modules

In [ ]:
from typing import Optional

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_experimental.autonomous_agents import BabyAGI
from langchain_openai import OpenAI, OpenAIEmbeddings

## Connect to the Vector Store

Depending on what vectorstore you use, this step may look different.

In [ ]:
%pip install faiss-cpu > /dev/null
%pip install google-search-results > /dev/null
from langchain.docstore import InMemoryDocstore
from langchain_community.vectorstores import FAISS

In [ ]:
import os

OPENAI_API_TOKEN = "sk-proj-MLEybJFAMNvXCzYw95mJT3BlbkFJz9e3O8NhaQ6bx6t1zD4U"
os.environ["OPENAI_API_KEY"] = OPENAI_API_TOKEN

In [ ]:
# Define your embedding model
embeddings_model = OpenAIEmbeddings()
# Initialize the vectorstore as empty
import faiss

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

## Define the Chains

BabyAGI relies on three LLM chains:
- Task creation chain to select new tasks to add to the list
- Task prioritization chain to re-prioritize tasks
- Execution Chain to execute the tasks


NOTE: in this notebook, the Execution chain will now be an agent.

Please create free SERP API key here: https://serpapi.com/

In [ ]:
SERP_API_TOKEN="5e23264377b1c44cbe1fceb128926ee4c435e373c5e138733b2ad9106b346538"
os.environ["SERPAPI_API_KEY"] = SERP_API_TOKEN

In [ ]:
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain.chains import LLMChain
from langchain_community.utilities import SerpAPIWrapper
from langchain_openai import OpenAI

todo_prompt = PromptTemplate.from_template(
    "You are a planner who is an expert at coming up with a todo list for a given objective. Come up with a todo list for this objective: {objective}"
)
todo_chain = LLMChain(llm=OpenAI(temperature=0), prompt=todo_prompt)
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="TODO",
        func=todo_chain.run,
        description="useful for when you need to come up with todo lists. Input: an objective to create a todo list for. Output: a todo list for that objective. Please be very clear what the objective is!",
    ),
]


prefix = """You are an AI who performs one task based on the following objective: {objective}. Take into account these previously completed tasks: {context}."""
suffix = """Question: {task}
{agent_scratchpad}"""
prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["objective", "task", "context", "agent_scratchpad"],
)

<ipython-input-9-88ab0a222a1a>:9: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  todo_chain = LLMChain(llm=OpenAI(temperature=0), prompt=todo_prompt)


In [ ]:
llm = OpenAI(temperature=0)
llm_chain = LLMChain(llm=llm, prompt=prompt)
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

<ipython-input-10-caaec25ad234>:4: LangChainDeprecationWarning: The class `ZeroShotAgent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~create_react_agent` instead.
  agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)


### Run the BabyAGI

Now it's time to create the BabyAGI controller and watch it try to accomplish your objective.

In [ ]:
OBJECTIVE = "Write a weather report for Athens, Greece today"

In [ ]:
# Logging of LLMChains
verbose = False
# If None, will keep on going forever
max_iterations: Optional[int] = 3
baby_agi = BabyAGI.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    task_execution_chain=agent_executor,
    verbose=verbose,
    max_iterations=max_iterations,
)

In [ ]:
baby_agi({"objective": OBJECTIVE})

<ipython-input-13-b5a7585b5beb>:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  baby_agi({"objective": OBJECTIVE})



*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list


> Entering new AgentExecutor chain...
Thought: I should create a todo list for the objective of writing a weather report for Athens, Greece today.
Action: TODO
Action Input: Write a weather report for Athens, Greece today.
Observation: 

1. Research current weather conditions in Athens, Greece.
2. Gather information on temperature, humidity, wind speed, and precipitation.
3. Check for any weather alerts or warnings in the area.
4. Look up the forecast for the rest of the day and the next few days.
5. Take note of any significant weather events or patterns in the region.
6. Determine the best format for the weather report (e.g. written, video, infographic).
7. Create a rough outline for the report, including sections for current conditions, forecast, and any additional information.
8. Begin writing the report, starting with an attention-grabbing headline.
9. Use clear and concise language to describe t

{'objective': 'Write a weather report for Athens, Greece today'}